In [3]:
pip install sparse_dot_topn

  Created wheel for sparse-dot-topn: filename=sparse_dot_topn-0.2.9-cp38-cp38-win_amd64.whl size=64610 sha256=549e9c4312cbc590a7e51abd02d915fbe5dc69e6cc3d7e3a88bf9fff99eecae2
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\4a\64\25\2e1cedbf7513fecda8d396a0431f3a1e217f45f1ab565e8ce8
Successfully built sparse-dot-topn
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct  # Leading Juice for us
import time
pd.set_option('display.max_colwidth', -1)


<ipython-input-4-1fecbd809729>:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [6]:
df1 = pd.read_csv('user_from_Aug')

FileNotFoundError: [Errno 2] No such file or directory: 'user_from_Aug'

In [4]:
# reading dataset as df
df =  pd.read_csv('room_type.csv')
# printing first five rows
df.head(5)

,RoomTypes
0,"Deluxe Room, 1 King Bed"
1,"Standard Room, 1 King Bed, Accessible"
2,"Grand Corner King Room, 1 King Bed"
3,"Suite, 1 King Bed (Parlor)"
4,"High-Floor Premium Room, 1 King Bed"


In [7]:
#  ngrams(here we are taking n = 3 thus 3-gram (trigrams ) as  most room types only contain two or three words
#  used for cleaning and removing some punctuation (dots, comma’s etc) i.e.((,-./)) from a string 
#  and generate and collect all n-grams of the string.  

 
def ngrams(string, n=3):

    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]





# Testing ngrams work for verification 

print('All 3-grams in "Deluxroom":')
ngrams('Deluxroom')


All 3-grams in "Deluxroom":


['Del', 'elu', 'lux', 'uxr', 'xro', 'roo', 'oom']

In [8]:
room_types = df['RoomTypes']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(room_types)

print(tf_idf_matrix[0])

  (0, 113)	0.1605304445953455
  (0, 29)	0.16141635519747793
  (0, 318)	0.22675894223265677
  (0, 452)	0.18208940066082585
  (0, 371)	0.17370536296468006
  (0, 140)	0.17886291412226296
  (0, 46)	0.18887063512800203
  (0, 99)	0.2517854902990057
  (0, 24)	0.23022653423352535
  (0, 412)	0.2609327210548879
  (0, 473)	0.15046703253147248
  (0, 480)	0.13094460588705156
  (0, 168)	0.1363445719787321
  (0, 62)	0.16141635519747793
  (0, 279)	0.28518161479728127
  (0, 612)	0.2882757113194961
  (0, 596)	0.2882757113194961
  (0, 409)	0.2882757113194961
  (0, 297)	0.2882757113194961
  (0, 120)	0.2882757113194961


In [10]:

# calculate the similarity between two vectors of TF-IDF values the Cosine Similarity is usually used.
# result matrix in a very sparse terms and Scikit-learn deals with this nicely by returning a sparse CSR matrix.

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [11]:
#  Top 10 with similarity above 0.8
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.0038623809814453125


In [12]:
# unpacks the resulting sparse matrix
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [13]:
# store the  matches into new dataframe called matched_df and 
# printing 10 samples
matches_df = get_matches_df(matches, room_types, top=200)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # For removing all exact matches
matches_df.sample(10)

,left_side,right_side,similairity
133,"Standard Room, Partial Ocean View (Waikiki Tower) - No Resort Fee","Standard Room, Ocean View (Waikiki Tower) - No Resort Fee",0.905160
40,"Club Room, City View (Club Lounge Access for 2 guests)","Club Room, Lake View (Club Lounge Access for 2 guests)",0.875506
175,Two Double Beds - Location Room (19th to 25th Floors),King Bed - Location Room (19th to 25th Floors),0.875607
27,"Premium Room, 2 Queen Beds","Premium Room, 1 Queen Bed",0.816652
94,"Deluxe Suite, 1 Bedroom","Suite, 1 Bedroom",0.813644
25,"Signature Room, 1 King Bed",Signature King,0.829876
6,"Grand Corner King Room, 1 King Bed",Grand Corner King Room,0.933038
42,"Club Room, Lake View (Club Lounge Access for 2 guests)","Club Room, City View (Club Lounge Access for 2 guests)",0.875506
191,One Bedroom Suite,One - Bedroom Suite,0.830120
158,Grand Corner King Room,"Grand Corner King Room, 1 King Bed",0.933038


In [14]:
# printing the matches in sorted order
matches_df.sort_values(['similairity'], ascending=False).head(10)


,left_side,right_side,similairity
121,"Room, Ocean View","Room, Ocean View",0.946453
138,"Room, Ocean View","Room, Ocean View",0.946453
6,"Grand Corner King Room, 1 King Bed",Grand Corner King Room,0.933038
158,Grand Corner King Room,"Grand Corner King Room, 1 King Bed",0.933038
167,King Room - Disability Access,Queen Room - Disability Access,0.919632
168,King Room - Disability Access,Queen Room - Disability Access,0.919632
135,"Standard Room, Ocean View (Waikiki Tower) - No Resort Fee","Standard Room, Partial Ocean View (Waikiki Tower) - No Resort Fee",0.905160
133,"Standard Room, Partial Ocean View (Waikiki Tower) - No Resort Fee","Standard Room, Ocean View (Waikiki Tower) - No Resort Fee",0.905160
175,Two Double Beds - Location Room (19th to 25th Floors),King Bed - Location Room (19th to 25th Floors),0.875607
177,King Bed - Location Room (19th to 25th Floors),Two Double Beds - Location Room (19th to 25th Floors),0.875607
